In [1]:
import sys
import nltk
import sklearn
import pandas
import numpy

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))

Python: 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]
NLTK: 3.6.1
Scikit-learn: 1.0.2
Pandas: 1.2.4
Numpy: 1.20.1


In [2]:
import warnings
warnings.filterwarnings('ignore')

# Load the dataset

In [3]:
import pandas as pd
import numpy as np

#load the dataset of sms messages
df= pd.read_table('SMSSpamCollection', header=None, encoding='utf-8')

In [4]:
# print useful information about the dataset
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5572 non-null   object
 1   1       5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None
      0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [5]:
# check the class distribution
classes = df[0]
print(classes.value_counts())

ham     4825
spam     747
Name: 0, dtype: int64


# Preprocessing of data

In [6]:
# convert class labels to binary values, 0=ham, 1=spam

from sklearn.preprocessing import LabelEncoder

encoder= LabelEncoder()
Y = encoder.fit_transform(classes)

print(classes[:10])
print(Y[:10])

0     ham
1     ham
2    spam
3     ham
4     ham
5    spam
6     ham
7     ham
8    spam
9    spam
Name: 0, dtype: object
[0 0 1 0 0 1 0 0 1 1]


In [7]:
# store the SMS message data
text_messages = df[1]
print(text_messages[:10])

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: 1, dtype: object


In [8]:
# use regular expression to replace email addresses, urls, phone  numbers, other numbers, symbols

# replace email address with 'emailaddr'
processed = text_messages.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$', 'emailaddr')

# replace urls with 'webaddress'
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','webaddress')

# replace money symbols with 'moneysymb'
processed = processed.str.replace(r'£|\$', 'moneysymb')

# replace 10 digit phone numbers with 'phonenumber'
processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$','phonenumbr')

# replace normal numbers with 'numbr'
processed = processed.str.replace(r'\d+(\.\d+)?','numbr')

In [9]:
# remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ')

# replaced whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ')

# remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')

In [10]:
# changing words to lower case = Hello, HELLO, hello are all the same word!
processed = processed.str.lower()
print(processed)

0       go until jurong point crazy available only in ...
1                                 ok lar joking wif u oni
2       free entry in numbr a wkly comp to win fa cup ...
3             u dun say so early hor u c already then say
4       nah i don t think he goes to usf he lives arou...
                              ...                        
5567    this is the numbrnd time we have tried numbr c...
5568                  will ü b going to esplanade fr home
5569    pity was in mood for that so any other suggest...
5570    the guy did some bitching but i acted like i d...
5571                            rofl its true to its name
Name: 1, Length: 5572, dtype: object


In [11]:
# remove stop words from the text messages

from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

In [12]:
# remove word stems using a Porter stemmer
ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(ps.stem(term) for term in x.split()))

In [13]:
print(processed)

0       go jurong point crazi avail bugi n great world...
1                                   ok lar joke wif u oni
2       free entri numbr wkli comp win fa cup final tk...
3                     u dun say earli hor u c alreadi say
4                    nah think goe usf live around though
                              ...                        
5567    numbrnd time tri numbr contact u u moneysymbnu...
5568                              ü b go esplanad fr home
5569                                    piti mood suggest
5570    guy bitch act like interest buy someth els nex...
5571                                       rofl true name
Name: 1, Length: 5572, dtype: object


In [14]:
from nltk.tokenize import word_tokenize

# creating a bag-ofwords
all_words = []

for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)

In [15]:
# print the total number words and the 15 most common words.
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))

Number of words: 6579
Most common words: [('numbr', 2648), ('u', 1207), ('call', 674), ('go', 456), ('get', 451), ('ur', 391), ('gt', 318), ('lt', 316), ('come', 304), ('moneysymbnumbr', 303), ('ok', 293), ('free', 284), ('day', 276), ('know', 275), ('love', 266)]


In [16]:
# use the 1500 most common words as features
word_features = list(all_words.keys())[:1500]

In [18]:
# define a find_features function
def find_features(message):
    words = set(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)
        
    return features

In [19]:
# let's see an example
features = find_features(processed[0])
for key, value in features.items():
    if value == True:
        print(key)

n
e
u
c
r
v
b
f
g
w
z
l


In [20]:
processed[0]

'go jurong point crazi avail bugi n great world la e buffet cine got amor wat'

In [21]:
features

{'go': False,
 'jurong': False,
 'point': False,
 'crazi': False,
 'avail': False,
 'bugi': False,
 'n': True,
 'great': False,
 'world': False,
 'la': False,
 'e': True,
 'buffet': False,
 'cine': False,
 'got': False,
 'amor': False,
 'wat': False,
 'ok': False,
 'lar': False,
 'joke': False,
 'wif': False,
 'u': True,
 'oni': False,
 'free': False,
 'entri': False,
 'numbr': False,
 'wkli': False,
 'comp': False,
 'win': False,
 'fa': False,
 'cup': False,
 'final': False,
 'tkt': False,
 'numbrst': False,
 'may': False,
 'text': False,
 'receiv': False,
 'question': False,
 'std': False,
 'txt': False,
 'rate': False,
 'c': True,
 'appli': False,
 'numbrovernumbr': False,
 'dun': False,
 'say': False,
 'earli': False,
 'hor': False,
 'alreadi': False,
 'nah': False,
 'think': False,
 'goe': False,
 'usf': False,
 'live': False,
 'around': False,
 'though': False,
 'freemsg': False,
 'hey': False,
 'darl': False,
 'week': False,
 'word': False,
 'back': False,
 'like': False,
 'fun'

In [22]:
# find features for all messages
messages = list(zip(processed, Y))

# define a seed for reproducibility
seed=1
np.random.seed = seed
np.random.shuffle(messages)

# call find_features function for each SMS messages
featuresets = [(find_features(text), label) for (text, label) in messages]

In [23]:
from sklearn import model_selection

training, testing = model_selection.train_test_split(featuresets, test_size=0.25, random_state=seed)

In [24]:
print('Training: {}'.format(len(training)))
print('Testing: {}'.format(len(testing)))

Training: 4179
Testing: 1393


## Scikit learn Classifier with nltk

In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [26]:
# Define models to train
names = ['K Nearest Neighbors', 'Decesion Tree', 'Random Forest', 'Logistic Regression', 'SGD Classifier', 'Naive Bayes', 'SVM']

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter=100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)
print(models)

In [27]:
# wrap models in nltk
from nltk.classify.scikitlearn import SklearnClassifier

for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing) * 100
    print('{}: Accuracy: {}'.format(name, accuracy))

K Nearest Neighbors: Accuracy: 90.59583632447954
Decesion Tree: Accuracy: 91.02656137832017
Random Forest: Accuracy: 91.31371141421393
Logistic Regression: Accuracy: 91.38549892318737
SGD Classifier: Accuracy: 88.15506101938263
Naive Bayes: Accuracy: 86.36037329504667
SVM: Accuracy: 90.52404881550609


In [35]:
# ensemble method - voting classifier
from sklearn.ensemble import VotingClassifier

# Define model to train
names = ['K Nearest Neighbors', 'Decesion Tree', 'Random Forest', 'Logistic Regression', 'SGD Classifier', 'Naive Bayes', 'SVM']

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter=100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))

nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = models, voting = 'hard', n_jobs=-1))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_ensemble, testing) * 100
print('Ensemble Method Accuracy: {}'.format(accuracy))

Ensemble Method Accuracy: 91.17013639626705


In [36]:
# make class label prediction for testng set
txt_features, labels = zip(*testing)

prediction = nltk_ensemble.classify_many(txt_features)

In [37]:
# print a confusion matrix and a classification report
print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index = [['actual', 'actual'], ['ham', 'spam']],
    columns= [['predicted', 'predicted'], ['ham', 'spam']])

              precision    recall  f1-score   support

           0       0.93      0.97      0.95      1203
           1       0.73      0.55      0.63       190

    accuracy                           0.91      1393
   macro avg       0.83      0.76      0.79      1393
weighted avg       0.91      0.91      0.91      1393



predicted     
                  ham spam
actual ham       1165   38
       spam        85  105